In [22]:
from PIL import Image
from os import listdir
import pandas as pd
import numpy as np
import re
from collections import defaultdict
import seaborn as sns
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D , Activation, Dropout ,  Flatten, Dense
from keras import backend as K 
from keras.preprocessing import image
import cv2
import itertools
import matplotlib.pyplot as plt
import os


from datetime import datetime
import time
# from module import *

from sklearn.datasets import fetch_openml
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.model_selection import learning_curve, train_test_split
from skimage.feature import hog
from skimage import data, exposure

from sklearn.cluster import MiniBatchKMeans
from sklearn.neural_network import MLPClassifier
import csv






In [120]:
def sift_desc(path):
    #reading the image using imread() function from cv2 module and converting it into gray image
    img = cv2.imread(path,0)
    equ = cv2.equalizeHist(img) 

    #creating a sift object and using detectandcompute() function to detect the keypoints and descriptor from the image
    sift = cv2.xfeatures2d.SIFT_create(500) # nombre de descripteur max recherché par image
    kp, desc = sift.detectAndCompute(equ, None)  # return key points and descriptors
    
    return desc,sift

In [24]:
def hog(img):
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1), visualize=True, channel_axis=-1)

# Extraction

In [121]:
img_dic=defaultdict(list)
directory='photos/dataset'
sift_all_desc=[]

for filename in listdir(directory):
    img_dic['name'].append(filename)
    #add target
    img_dic['y'].append(re.findall(r"[^\W\d]+",filename)[0])
    # load image
    img_data = Image.open(directory + '/' + filename)
    img_dic['img_origine'].append(img_data)

    # get description
    desc,sift=sift_desc(directory + '/' + filename)
    img_dic['desc'].append(desc)
    img_dic['all_desc'].extend(desc)



In [75]:
# k = 400 #round(np.sqrt(len(img_dic['sift_all_desc'])))
# k=40

k = round(np.sqrt(len(img_dic['all_desc'])))
# print(k)
batch_size = np.size(img_dic['name']) * 3
kmeans = MiniBatchKMeans(n_clusters=k, batch_size=batch_size, verbose=1).fit(img_dic['all_desc'])


40
Init 1/3 with method k-means++
Inertia for init 1/3: 451028523.0
Init 2/3 with method k-means++
Inertia for init 2/3: 456715176.0
Init 3/3 with method k-means++
Inertia for init 3/3: 455377123.0
Minibatch step 1/3249: mean batch inertia: 126156.105
Minibatch step 2/3249: mean batch inertia: 83397.95219851499, ewa inertia: 83397.95219851499
Minibatch step 3/3249: mean batch inertia: 83440.36863776411, ewa inertia: 83400.56311039491
Minibatch step 4/3249: mean batch inertia: 84077.5412650026, ewa inertia: 83442.23398936538
Minibatch step 5/3249: mean batch inertia: 83860.43802630884, ewa inertia: 83467.97622298078
Minibatch step 6/3249: mean batch inertia: 82915.62045499575, ewa inertia: 83433.97638088887
Minibatch step 7/3249: mean batch inertia: 83244.61306923558, ewa inertia: 83422.3202652703
Minibatch step 8/3249: mean batch inertia: 82525.837736932, ewa inertia: 83367.13796037131
Minibatch step 9/3249: mean batch inertia: 82512.25522031558, ewa inertia: 83314.51629901883
Minibatc

# Histogram

In [111]:
kmeans.verbose = False

histo_list = []

for desc in img_dic['desc']:

    histo = np.zeros(k)
    nkp = len(desc)

    for d in desc:
        idx = kmeans.predict([d])
        histo[idx] += 1/nkp # Because we need normalized histograms, I prefere to add 1/nkp directly

    histo_list.append(histo)
features = np.asarray(histo_list,dtype=object)
# feature    = np.concatenate(temps, axis=0)
    

In [115]:
pd.DataFrame(features)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.040541,0.067568,0.0,0.027027,0.0,0.013514,0.013514,0.013514,0.0,0.027027,...,0.040541,0.013514,0.013514,0.027027,0.0,0.027027,0.0,0.013514,0.0,0.0
1,0.01,0.02,0.01,0.04,0.01,0.03,0.01,0.03,0.02,0.03,...,0.04,0.04,0.02,0.07,0.02,0.05,0.02,0.0,0.01,0.04
2,0.0,0.0,0.13,0.02,0.0,0.01,0.02,0.02,0.02,0.02,...,0.04,0.0,0.02,0.07,0.02,0.03,0.01,0.02,0.0,0.01
3,0.0,0.02,0.0,0.01,0.08,0.0,0.01,0.03,0.01,0.05,...,0.0,0.04,0.05,0.0,0.05,0.03,0.03,0.04,0.05,0.06
4,0.0,0.039604,0.039604,0.059406,0.029703,0.0,0.019802,0.009901,0.039604,0.029703,...,0.029703,0.0,0.049505,0.029703,0.039604,0.039604,0.019802,0.0,0.009901,0.019802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.0,0.05,0.07,0.0,0.13,0.0,0.0,0.06,0.0,0.01,...,0.0,0.01,0.07,0.0,0.03,0.0,0.03,0.04,0.15,0.1
396,0.0,0.0,0.02,0.0,0.0,0.0,0.01,0.08,0.03,0.05,...,0.0,0.0,0.02,0.01,0.0,0.0,0.07,0.03,0.0,0.05
397,0.009901,0.0,0.019802,0.0,0.0,0.019802,0.019802,0.039604,0.039604,0.009901,...,0.009901,0.029703,0.049505,0.19802,0.009901,0.009901,0.019802,0.0,0.0,0.0
398,0.02,0.0,0.07,0.0,0.03,0.09,0.0,0.03,0.06,0.03,...,0.0,0.02,0.01,0.03,0.0,0.0,0.01,0.01,0.01,0.03


In [116]:
pd.DataFrame(histo_list)

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,0.040541,0.067568,0.000000,0.027027,0.000000,0.013514,0.013514,0.013514,0.000000,0.027027,...,0.040541,0.013514,0.013514,0.027027,0.000000,0.027027,0.000000,0.013514,0.000000,0.000000
1,0.010000,0.020000,0.010000,0.040000,0.010000,0.030000,0.010000,0.030000,0.020000,0.030000,...,0.040000,0.040000,0.020000,0.070000,0.020000,0.050000,0.020000,0.000000,0.010000,0.040000
2,0.000000,0.000000,0.130000,0.020000,0.000000,0.010000,0.020000,0.020000,0.020000,0.020000,...,0.040000,0.000000,0.020000,0.070000,0.020000,0.030000,0.010000,0.020000,0.000000,0.010000
3,0.000000,0.020000,0.000000,0.010000,0.080000,0.000000,0.010000,0.030000,0.010000,0.050000,...,0.000000,0.040000,0.050000,0.000000,0.050000,0.030000,0.030000,0.040000,0.050000,0.060000
4,0.000000,0.039604,0.039604,0.059406,0.029703,0.000000,0.019802,0.009901,0.039604,0.029703,...,0.029703,0.000000,0.049505,0.029703,0.039604,0.039604,0.019802,0.000000,0.009901,0.019802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,0.000000,0.050000,0.070000,0.000000,0.130000,0.000000,0.000000,0.060000,0.000000,0.010000,...,0.000000,0.010000,0.070000,0.000000,0.030000,0.000000,0.030000,0.040000,0.150000,0.100000
396,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.080000,0.030000,0.050000,...,0.000000,0.000000,0.020000,0.010000,0.000000,0.000000,0.070000,0.030000,0.000000,0.050000
397,0.009901,0.000000,0.019802,0.000000,0.000000,0.019802,0.019802,0.039604,0.039604,0.009901,...,0.009901,0.029703,0.049505,0.198020,0.009901,0.009901,0.019802,0.000000,0.000000,0.000000
398,0.020000,0.000000,0.070000,0.000000,0.030000,0.090000,0.000000,0.030000,0.060000,0.030000,...,0.000000,0.020000,0.010000,0.030000,0.000000,0.000000,0.010000,0.010000,0.010000,0.030000


# Training

In [60]:
species=list(set(img_dic['y']))
species=sorted(species)
species=np.array(species)

In [61]:
species

array(['cloudy', 'rain', 'shine', 'sunrise'], dtype='<U7')

In [118]:
X = np.array(histo_list)
Y = []

# It's a way to convert species name into an integer
for s in img_dic['y']:
    Y.append(np.min(np.nonzero(species == s)))

mlp = MLPClassifier(verbose=0, max_iter=600000,)
mlp.fit(X, Y)

MLPClassifier(max_iter=600000, verbose=0)

In [119]:
mlp.score(X,Y)

0.9975

# Prediction

In [65]:
result_file = open("sift.csv", "w")
result_file_obj = csv.writer(result_file)
result_file_obj.writerow(np.append("id", species))

for desc in img_dic['desc']:
    
    x = np.zeros(k)
    nkp = np.size(desc)

    for d in desc:
        idx = kmeans.predict([d])
        x[idx] += 1/nkp

    res = mlp.predict_proba([x])
    row = []
    row.append(filename)

    for e in res[0]:
        row.append(e)

    result_file_obj.writerow(row)

result_file.close()

In [ ]:
mlp.score(X,Y)

0.25